In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
from fbprophet.plot import plot_plotly
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import *
from fbprophet import Prophet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/demand-forecasting-kernels-only/train.csv')
test=pd.read_csv("../input/demand-forecasting-kernels-only/test.csv")
sample=pd.read_csv("../input/demand-forecasting-kernels-only/sample_submission.csv")

In [ ]:
train["year"] = pd.to_datetime(train["date"]).dt.year
train["month"] = pd.to_datetime(train["date"]).dt.month
train["day"] = pd.to_datetime(train["date"]).dt.day
train['dayOfWeek'] =  pd.to_datetime(train["date"]).dt.dayofweek

In [ ]:
test["year"] = pd.to_datetime(test["date"]).dt.year
test["month"] = pd.to_datetime(test["date"]).dt.month
test["day"] = pd.to_datetime(test["date"]).dt.day
test['dayOfWeek'] =  pd.to_datetime(test["date"]).dt.dayofweek

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
df=train[train['store']==1][train['item']==1]

In [ ]:
df.shape

In [ ]:
df["sales"] = np.log1p(df["sales"])

In [ ]:
df = df[["date", "sales"]]

In [ ]:
df=df.rename(columns={"date":"ds","sales":"y"})

In [ ]:
df.ds=pd.to_datetime(df.ds,format='%Y-%m')

In [ ]:
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
preds= forecast['yhat'][:-365]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.ds,preds,color='red',label=' Predictions')
plt.plot(df.ds,df.y,color='blue',label='Actual')
plt.legend()
plt.show()

In [ ]:
r2 = round(r2_score(df["y"], preds), 3)
mse = round(mean_squared_error(df["y"], preds), 3)
mae = round(mean_absolute_error(df["y"], preds), 3)
print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

In [ ]:
fig1 = m.plot(forecast)
fig1.set_size_inches(15,6)

In [ ]:
fig2 = m.plot_components(forecast)
plt.show()

**My analysis: We see sales drop from Sunday to Monday so there must be a holiday effect in the sales data. There's a peak in July so this may be due to seasonal sales or summer festivities.**

**Next step, try excluding change points and include holiday effects. Let's count NFL playoffs as a holiday. And add an extra regressor for NFL sundays.**

In [ ]:
playoffs = ['2013-07-12', '2014-07-12', '2014-07-19',
                 '2014-07-02', '2015-07-11', '2016-07-17',
                 '2016-07-24', '2016-07-07','2016-07-24']
superbowl = ['2013-01-01', '2013-12-25', '2014-01-01', '2014-12-25','2015-01-01', '2015-12-25','2016-01-01', '2016-12-25',
                '2017-01-01', '2017-12-25']

playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(playoffs),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(superbowl),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

In [ ]:
df["dow"] = pd.to_datetime(train["date"]).dt.day_name() # day of week
df.head()

In [ ]:
def nfl_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and (date.month > 8 or date.month < 2):
        return 1
    else:
        return 0

In [ ]:
df =df[["ds", 'y']]
df.head()

In [ ]:
df["nfl_sunday"] = df['ds'].apply(nfl_sunday)
df

In [ ]:
m = Prophet(holidays=holidays, holidays_prior_scale=0.5,
            yearly_seasonality=4,  interval_width=0.95,
            changepoint_prior_scale=0.006, daily_seasonality=True)
m.add_regressor('nfl_sunday')
m.add_seasonality(name='daily', period=60, fourier_order=5)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=90, freq="D") # Daily frequency
future['nfl_sunday'] = future['ds'].apply(nfl_sunday)
future.head()

In [ ]:
forecast=m.predict(future)

In [ ]:
preds= forecast['yhat'][:-90]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.ds,preds,color='red',label=' Predictions')
plt.plot(df.ds,df.y,color='blue',label='Actual')
plt.legend()
plt.show()

In [ ]:
r2 = round(r2_score(df["y"], preds), 3)
mse = round(mean_squared_error(df["y"], preds), 3)
mae = round(mean_absolute_error(df["y"], preds), 3)
print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)
plt.show()

In [ ]:
future.tail()

In [ ]:
ps1i1 = forecast[["ds"]]
ps1i1["forecast"] = np.expm1(forecast["yhat"])
ps1i1["yearmonth"] = pd.to_datetime(ps1i1["ds"]).dt.to_period("M")
ps1i1.head()

In [ ]:
def smape(outsample, forecast):
    num = np.abs(outsample-forecast)
    denom = np.abs(outsample) + np.abs(forecast)
    return (num/denom)/2

df["ds"] = pd.to_datetime(df["ds"])

In [ ]:
ps1i1["ds"] = pd.to_datetime(ps1i1["ds"])

In [ ]:
train_predict = df.merge(ps1i1)

In [ ]:
smape_err = smape(train_predict["y"], train_predict["forecast"])
smape_err = smape_err[~np.isnan(smape_err)]
np.mean(smape_err)

In [ ]:
def make_prediction(df):
    
    playoffs = ['2013-07-12', '2014-07-12', '2014-07-19',
                 '2014-07-02', '2015-07-11', '2016-07-17',
                 '2016-07-24', '2016-07-07','2016-07-24']
    superbowl = ['2013-01-01', '2013-12-25', '2014-01-01', '2014-12-25','2015-01-01', '2015-12-25','2016-01-01', '2016-12-25',
                    '2017-01-01', '2017-12-25']

    playoffs = pd.DataFrame({
      'holiday': 'playoff',
      'ds': pd.to_datetime(playoffs),
      'lower_window': 0,
      'upper_window': 1,
    })
    superbowls = pd.DataFrame({
      'holiday': 'superbowl',
      'ds': pd.to_datetime(superbowl),
      'lower_window': 0,
      'upper_window': 1,
    })
    holidays = pd.concat((playoffs, superbowls))
    
    m = Prophet(holidays=holidays, holidays_prior_scale=0.5,
            yearly_seasonality=4,  interval_width=0.95,
            changepoint_prior_scale=0.006, daily_seasonality=True)
    m.add_seasonality(name='daily', period=60, fourier_order=5)
    m.fit(df)
    future = m.make_future_dataframe(periods=90)
    forecast = m.predict(future)
    return forecast